# 3. Feature Engineering

In [1]:
# Import python packages
import numpy as np
import pandas as pd

## 3.1 Business Features

In [2]:
# Read business data and checkin data
business = pd.read_csv('clean_business.csv')
business.shape

(2493, 13)

In [3]:
# Reveal first few rows 
business.head()

attributes             business_id  \
0  {'RestaurantsTableService': False, 'GoodForMea...  rDMptJYWtnMhpQu_rRXHng   
1  {'RestaurantsTableService': True, 'GoodForMeal...  1WBkAuQg81kokZIPMpn9Zg   
2  {'RestaurantsTableService': False, 'GoodForMea...  iPa__LOhse-hobC2Xmp-Kw   
3  {'BusinessAcceptsCreditCards': True, 'Business...  YhV93k9uiMdr3FlV4FHjwA   
4  {'RestaurantsTableService': True, 'GoodForMeal...  QkG3KUXwqZBW18A9k1xqCA   

                                          categories     city   latitude  \
0            ['Fast Food', 'Burgers', 'Restaurants']  Phoenix  33.607070   
1                         ['Burgers', 'Restaurants']  Phoenix  33.607310   
2            ['Restaurants', 'Burgers', 'Fast Food']  Phoenix  33.508765   
3  ['Marketing', "Men's Clothing", 'Restaurants',...  Phoenix  33.449967   
4  ['American (Traditional)', 'Restaurants', 'Sea...  Phoenix  33.478735   

    longitude                          name  postal_code  review_count  stars  \
0 -112.064382                    McDonald's        85022            10    1.0   
1 -112.063404  Charr An American Burger Bar        85022           232    3.0   
2 -112.046240                    McDonald's        85016            34    3.0   
3 -112.070222               Caviness Studio        85001             4    5.0   
4 -112.221379                   Red Lobster        85035            37    2.5   

  state  isBankrupt           county  
0    AZ           0  Maricopa County  
1    AZ           0  Maricopa County  
2    AZ           0  Maricopa County  
3    AZ           0  Maricopa County  
4    AZ           0  Maricopa County

In [4]:
# Make a copy of raw dataset
business_df = business.copy()

## 3.1.1 Create isChain

We define `isChain` variable by checking whether the name of the restaurant has appeared more than once. 

In [5]:
# Create new feature chain
chain_dict = business_df['name'].value_counts().to_dict()
business_df['isChain'] = business_df['name'].apply(lambda x: int(chain_dict[x] > 1))

## 3.1.2 Create Neighborhood Density (Do this later)

## 3.1.3 Extract Business Categories

In [6]:
# Check categories type
type(business_df['categories'][0])

str

In [7]:
# Convert categories string into list
business_df['categories'] = business_df['categories'].apply(lambda x: eval(x))

In [21]:
# Extract all unique categories
categories = np.unique([item for row in business_df['categories'] for item in row])
print("Number of unique categories:", len(categories))

Number of unique categories: 271


## Restaurant categories I - Ethnicity

### a) Extract all ethnics

In [9]:
# All ethnics found in the dataset
all_ethnics = ['Afghan', 'African', 'American (New)', 'American (Traditional)', 
               'Arabian', 'Argentine','Armenian', 'Asian Fusion', 'British', 
               'Cajun/Creole', 'Cambodian', 'Cantonese', 'Caribbean', 'Chinese', 
               'Indian', 'Cuban', 'Empanadas', 'Ethiopian', 'Filipino', 'French', 
               'Greek', 'Hawaiian', 'Irish', 'Italian', 'Japanese', 'Korean', 
               'Kosher', 'Halal', 'Latin American', 'Lebanese', 'Mediterranean', 
               'Mexican', 'Middle Eastern', 'Modern European', 'Mongolian',
               'Moroccan', 'Pakistani', 'Persian/Iranian', 'Peruvian', 'Puerto Rican', 
               'Russian', 'Salvadoran', 'Southern', 'Spanish', 'Szechuan', 
               'Thai', 'Tex-Mex', 'Vietnamese', 'Turkish', 'Ukrainian', 'Uzbek']

In [10]:
# Extract all ethnics from categories
business_df['ethnics'] = business_df['categories'].apply(lambda row: [x for x in row if x in all_ethnics])

In [11]:
# Create a dictionary to count the occurence of each ethnic
ethnics_dict = dict(zip(all_ethnics, [sum([x in row for row in business_df['ethnics']]) for x in all_ethnics]))
print('Total Number of Ethnics:', len(ethnics_dict))

Total Number of Ethnics: 51


In [12]:
# Convert dictionary to dataframe
ethnics_df = pd.DataFrame(list(ethnics_dict.items()), columns=['Ethnics','Count']) \
                .sort_values('Count', ascending = False) \
                .reset_index() \
                .drop('index', axis=1)
ethnics_df.head()

Ethnics  Count
0                 Mexican    490
1  American (Traditional)    339
2          American (New)    246
3                 Italian    190
4                 Chinese    144

### b) Combine ethnicity into smaller groups

Since there are too many ethnics categories in the dataset, we combine these categories into smaller groups.

In [13]:
# Combine ethnics into fewer categories
ethnic_dict = {
    'African': ['African', 'Ethiopian', 'Moroccan'],
    'American': ['American (New)', 'American (Traditional)', 'Hawaiian', 'Southern'],
    'AsiaContinent': ['Mongolian', 'Russian', 'Ukrainian', 'Uzbek'],
    'Chinese': ['Cantonese', 'Chinese', 'Szechuan'],
    'EastAsia': ['Japanese', 'Korean', 'Asian Fusion'],
    'European': ['British', 'French', 'Greek', 'Irish', 'Kosher', 'Modern European', 'Cajun/Creole', 'Spanish'],
    'LatinAmerica': ['Argentine', 'Caribbean', 'Cuban', 'Empanadas', 
                     'Latin American', 'Peruvian', 'Puerto Rican', 'Salvadoran'],
    'Mexican': ['Mexican','Tex-Mex'],
    'MiddleEast': ['Arabian', 'Armenian', 'Halal', 'Lebanese', 'Middle Eastern', 
                   'Turkish','Mediterranean', 'Afghan', 'Pakistani', 'Persian/Iranian'],
    'SouthEastAsia': ['Cambodian', 'Filipino', 'Laotian', 'Thai', 'Vietnamese','Indian']
}

In [14]:
# Merge ethnics into smaller categories
merge_ethnic = lambda row: np.unique([ethnic if x in ethnic_dict[ethnic] else x for x in row])
for ethnic in ethnic_dict.keys():
    business_df['ethnics'] = business_df['ethnics'].apply(merge_ethnic)

In [15]:
# Update dictionary
ethnics = ethnic_dict.keys()
ethnics_dict = dict(zip(ethnics, [sum([x in row for row in business_df['ethnics'] if len(row) > 0]) for x in ethnics]))
print('Total Number of Ethnics:', len(ethnics_dict))

Total Number of Ethnics: 10


In [16]:
# Print dictionary
ethnics_dict

{'African': 9,
 'American': 542,
 'AsiaContinent': 5,
 'Chinese': 144,
 'EastAsia': 117,
 'European': 85,
 'LatinAmerica': 37,
 'Mexican': 496,
 'MiddleEast': 92,
 'SouthEastAsia': 78}

### c) Combine less frequent ethnic group

We will group ethnics that have very less than 50 occurence into `Other_Ethnic`

In [17]:
other_ethnic = [x for x in ethnics_dict.keys() if ethnics_dict[x] < 50]
other_ethnic

['African', 'AsiaContinent', 'LatinAmerica']

In [18]:
# Update feature ethnics
convert_to_other = lambda row: np.unique(['Other_Ethnic' if x in other_ethnic else x for x in row])
business_df['ethnics'] = business_df['ethnics'].apply(convert_to_other)

In [19]:
# Update dictionary
ethnics = [x for x in ethnics_dict.keys() if x not in other_ethnic] + ['Other_Ethnic']
ethnics_dict = dict(zip(ethnics, [sum([x in row for row in business_df['ethnics'] if len(row) > 0]) for x in ethnics]))
print('Total Number of Ethnics:', len(ethnics_dict))

Total Number of Ethnics: 8


In [20]:
# Convert dictionary to dataframe
ethnics_df = pd.DataFrame(list(ethnics_dict.items()), columns=['Ethnics','Count']) \
                .sort_values('Count',ascending = False) \
                .reset_index() \
                .drop('index', axis=1)
ethnics_df

Ethnics  Count
0       American    542
1        Mexican    496
2        Chinese    144
3       EastAsia    117
4     MiddleEast     92
5       European     85
6  SouthEastAsia     78
7   Other_Ethnic     51